# Database to Python

In [1]:
import time

In [88]:
import psycopg2
from sqlalchemy import create_engine
import pandas as pd
from datetime import datetime


In [3]:
databse = 'gfs_data'
user = 'gfs'
password = 'pollenmelb'
host="127.0.0.1"

conn_string = f"postgresql://{user}:{password}@{host}/{databse}"
engine = create_engine(conn_string)

In [4]:
dbConnection = engine.connect()

In [20]:
df = pd.read_sql("select * from \"weatherraw\" where coord[0] = 145.0 and coord[1] = -38", dbConnection)# pd.set_option('display.expand_frame_repr', False);
x= df.sort_values(by = 'datetime')


In [62]:
x.head()

,datetime,t,t_2m,msl,u_10m,v_10m,hum_atmos,pwat,coord,date
694,2000-01-01 00:00:00,288.6,288.3,101740.0,-1.6,3.500000,32.0,12.2,"(145,-38)",2000-01-01
8643,2000-01-01 06:00:00,290.8,290.3,101654.0,-0.1,6.000000,31.0,14.3,"(145,-38)",2000-01-01
7466,2000-01-01 12:00:00,284.0,284.0,101903.0,-1.2,3.400000,29.0,13.3,"(145,-38)",2000-01-01
3150,2000-01-01 18:00:00,280.1,280.4,101806.0,-0.9,2.300000,33.0,14.9,"(145,-38)",2000-01-01
8828,2000-01-02 00:00:00,292.3,291.4,101842.0,-1.1,1.499999,28.0,13.0,"(145,-38)",2000-01-02


## Add missing vals to db

In [108]:
mainfiles = []
for date in missing:
    files = []
    vals = date.split('-')
    if int(vals[0])<2008:
        grbval = 'grib1'
    else:
        grbval = 'grib2'
    date1 = '{}/{}/{}.{}/fnl_{}_00_00.{}'.format(grbval, vals[0], vals[0], vals[1], vals[0]+vals[1]+vals[2], grbval)
    date2 = '{}/{}/{}.{}/fnl_{}_06_00.{}'.format(grbval, vals[0], vals[0], vals[1], vals[0]+vals[1]+vals[2], grbval)
    date3 = '{}/{}/{}.{}/fnl_{}_12_00.{}'.format(grbval, vals[0], vals[0], vals[1], vals[0]+vals[1]+vals[2], grbval)
    date4 = '{}/{}/{}.{}/fnl_{}_18_00.{}'.format(grbval, vals[0], vals[0], vals[1], vals[0]+vals[1]+vals[2], grbval)
    if vals[1] == '12':
        date5 = '{}/{}/{}.{}/fnl_{}_00_00.{}'.format(grbval, vals[0], vals[0], vals[1], vals[0]+vals[1]+str(int(vals[2])+1), grbval)
    else:
        date5 = '{}/{}/{}.{}/fnl_{}_00_00.{}'.format(grbval, vals[0], vals[0], str(int(vals[1])%12 + 1).zfill(2), vals[0]+str(int(vals[1])%12 + 1).zfill(2)+'01', grbval)
    files.append(date1)
    files.append(date2)
    files.append(date3)
    files.append(date4)
    files.append(date5)
    mainfiles.append(files)

In [113]:
import sys, os
import requests
import numpy as np
import pandas as pd
import pygrib
from matplotlib import pyplot as plt
from matplotlib import colors
import numpy as np
from mpl_toolkits.basemap import Basemap, addcyclic
from datetime import datetime
import psycopg2
from sqlalchemy import create_engine


%matplotlib inline

In [135]:
# setup to download grib files from RDA archive

try:
    import getpass
    input = getpass.getpass
except:
    try:
        input = raw_input
    except:
        pass

In [136]:
# user specific data to login

pswd = input('password:')
values = {'email' : 'nandlalm@student.unimelb.edu.au', 'passwd' : pswd, 'action' : 'login'}
login_url = 'https://rda.ucar.edu/cgi-bin/login'

In [137]:
ret = requests.post(login_url, data=values)
if ret.status_code != 200:
    print('Bad Authentication')
    print(ret.text)
    exit(1)

In [117]:
# file path on RDA catalogue

dspath = 'https://rda.ucar.edu/data/ds083.2/'

In [118]:
# destinantion path

save_dir = '/pollensource/gfs_archive/'

In [119]:
# bounding box for Australia

min_lat = -43.835
max_lat = -9.796    
min_lon = 112.500
max_lon = 154.688


In [120]:
def filterAus(file):
    lat_filter = (file["lat"] >= min_lat) & (file["lat"] <= max_lat)
    lon_filter = (file["lon"] >= min_lon) & (file["lon"] <= max_lon)
    filtered = file.loc[lat_filter & lon_filter]
    return filtered.reset_index(drop=True)


In [121]:
# extract relevant data from the grib files

def getData(filename):

    fileAddress = '/pollensource/gfs_archive/' + filename
    myfile = pygrib.open(fileAddress)

    # get surface level temperature data
    grb_temp = myfile.select(shortName='t', typeOfLevel = 'surface')
    lat = grb_temp[0].latitudes
    lon = grb_temp[0].longitudes
    vals = grb_temp[0].values.reshape(lon.shape[0],)
    date = grb_temp[0].validDate
    
    df = pd.DataFrame(None, index=range(65160), columns=['datetime', 'lat', 'lon', 't'])  # range hard-coded for now
    df['datetime'] = date
    df['lat'] = lat
    df['lon'] = lon
    df['t'] = vals
    df['date'] = pd.to_datetime(df['datetime']).dt.date
    
    # get 2m level temperature data
    grb_temp_2m = myfile.select(shortName='2t')
    vals_2t = grb_temp_2m[0].values.reshape(lon.shape[0],)
    df['t_2m'] = vals_2t

    # get mean sea level pressure
    try:
        grb_msl = myfile.select(shortName='msl')
    except ValueError:
        grb_msl = myfile.select(shortName='prmsl')
    vals_msl = grb_msl[0].values.reshape(lon.shape[0],)
    df['msl'] = vals_msl

    # get 10m u component of wind
    grb_10u = myfile.select(shortName='10u')
    vals_10u = grb_10u[0].values.reshape(lon.shape[0],)
    df['u_10m'] = vals_10u

    # get 10m v component of wind
    grb_10v = myfile.select(shortName='10v')
    vals_10v = grb_10v[0].values.reshape(lon.shape[0],)
    df['v_10m'] = vals_10v

    # get the relative humidity
    try:
        grb_r = myfile.select(shortName='r', typeOfLevel = 'entireAtmosphere')
    except ValueError:
        grb_r = myfile.select(shortName='r', typeOfLevel = 'atmosphereSingleLayer')

    vals_r = grb_r[0].values.reshape(lon.shape[0],)
    df['hum_atmos'] = vals_r

    # get precipitable water
    grb_pwat = myfile.select(shortName='pwat')
    vals_pwat = grb_pwat[0].values.reshape(lon.shape[0],)
    df['pwat'] = vals_pwat

    data  = filterAus(df)
    # print(data)
    # data['coord'] = (data['lon'], data['lat'])
    data['coord'] = list(zip(data.lon, data.lat))
    data.coord = data.coord.astype(str)

    data = data.drop(['lat', 'lon'], axis = 1)

    return data

In [123]:
def addToDb(df):

    databse = 'gfs_data'
    user = 'gfs'
    password = 'pollenmelb'
    host="127.0.0.1"

    conn_string = f"postgresql://{user}:{password}@{host}/{databse}"
    engine = create_engine(conn_string)
    
    table_name = 'weather'
    
    if_exists = 'append'

    #Write the data to postgres

    with engine.connect() as con:
        df.to_sql(
            name=table_name.lower(), 
            con=con, 
            if_exists=if_exists,
            index = False
        )
    

In [124]:
# aggregate the hourly data and create variables

def aggregateData9am(dataf):

    grouped_t = dataf.groupby(['coord']).agg({'t': ['mean', 'min', 'max', np.std]})
    grouped_t.columns = ['t_mean_9am', 't_min_9am', 't_max_9am', 't_sd_9am']
    grouped_t = grouped_t.reset_index()

    grouped_t_2m = dataf.groupby(['coord']).agg({'t_2m': ['mean', 'min', 'max', np.std]})
    grouped_t_2m.columns = ['t_2m_mean_9am', 't_2m_min_9am', 't_2m_max_9am', 't_2m_sd_9am']
    grouped_t_2m = grouped_t_2m.reset_index()
    
    grouped_msl = dataf.groupby(['coord']).agg({'msl': ['mean', 'min', 'max', np.std]})
    grouped_msl.columns = ['msl_mean_9am', 'msl_min_9am', 'msl_max_9am', 'msl_sd_9am']
    grouped_msl = grouped_msl.reset_index()

    grouped_hum_atmos = dataf.groupby(['coord']).agg({'hum_atmos': ['mean', 'min', 'max', np.std]})
    grouped_hum_atmos.columns = ['hum_atmos_mean_9am', 'hum_atmos_min_9am', 'hum_atmos_max_9am', 'hum_atmos_sd_9am']
    grouped_hum_atmos = grouped_hum_atmos.reset_index()

    grouped_u_10m = dataf.groupby(['coord']).agg({'u_10m': ['mean', 'min', 'max', np.std]})
    grouped_u_10m.columns = ['u_10m_mean_9am', 'u_10m_min_9am', 'u_10m_max_9am', 'u_10m_sd_9am']
    grouped_u_10m = grouped_u_10m.reset_index()

    grouped_v_10m = dataf.groupby(['coord']).agg({'v_10m': ['mean', 'min', 'max', np.std]})
    grouped_v_10m.columns = ['v_10m_mean_9am', 'v_10m_min_9am', 'v_10m_max_9am', 'v_10m_sd_9am']
    grouped_v_10m = grouped_v_10m.reset_index()

    grouped_pwat = dataf.groupby(['coord']).agg({'pwat': ['mean', 'min', 'max', np.std]})
    grouped_pwat.columns = ['pwat_mean_9am', 'pwat_min_9am', 'pwat_max_9am', 'pwat_sd_9am']
    grouped_pwat = grouped_pwat.reset_index()

    agg_data = grouped_t.merge(grouped_t_2m, how = 'left', on = 'coord')
    agg_data = agg_data.merge(grouped_msl, how = 'left', on = 'coord')
    agg_data = agg_data.merge(grouped_hum_atmos, how = 'left', on = 'coord')
    agg_data = agg_data.merge(grouped_u_10m, how = 'left', on = 'coord')
    agg_data = agg_data.merge(grouped_v_10m, how = 'left', on = 'coord')
    agg_data = agg_data.merge(grouped_pwat, how = 'left', on = 'coord')

    dateval = min(dataf.date.unique())
    agg_data['date'] = dateval
    


    return agg_data

In [126]:
# aggregate the hourly data and create variables

def aggregateData4pm(dataf):

    grouped_t = dataf.groupby(['coord']).agg({'t': ['mean', 'min', 'max', np.std]})
    grouped_t.columns = ['t_mean_4pm', 't_min_4pm', 't_max_4pm', 't_sd_4pm']
    grouped_t = grouped_t.reset_index()

    grouped_t_2m = dataf.groupby(['coord']).agg({'t_2m': ['mean', 'min', 'max', np.std]})
    grouped_t_2m.columns = ['t_2m_mean_4pm', 't_2m_min_4pm', 't_2m_max_4pm', 't_2m_sd_4pm']
    grouped_t_2m = grouped_t_2m.reset_index()

    grouped_msl = dataf.groupby(['coord']).agg({'msl': ['mean', 'min', 'max', np.std]})
    grouped_msl.columns = ['msl_mean_4pm', 'msl_min_4pm', 'msl_max_4pm', 'msl_sd_4pm']
    grouped_msl = grouped_msl.reset_index()

    grouped_hum_atmos = dataf.groupby(['coord']).agg({'hum_atmos': ['mean', 'min', 'max', np.std]})
    grouped_hum_atmos.columns = ['hum_atmos_mean_4pm', 'hum_atmos_min_4pm', 'hum_atmos_max_4pm', 'hum_atmos_sd_4pm']
    grouped_hum_atmos = grouped_hum_atmos.reset_index()

    grouped_u_10m = dataf.groupby(['coord']).agg({'u_10m': ['mean', 'min', 'max', np.std]})
    grouped_u_10m.columns = ['u_10m_mean_4pm', 'u_10m_min_4pm', 'u_10m_max_4pm', 'u_10m_sd_4pm']
    grouped_u_10m = grouped_u_10m.reset_index()

    grouped_v_10m = dataf.groupby(['coord']).agg({'v_10m': ['mean', 'min', 'max', np.std]})
    grouped_v_10m.columns = ['v_10m_mean_4pm', 'v_10m_min_4pm', 'v_10m_max_4pm', 'v_10m_sd_4pm']
    grouped_v_10m = grouped_v_10m.reset_index()

    grouped_pwat = dataf.groupby(['coord']).agg({'pwat': ['mean', 'min', 'max', np.std]})
    grouped_pwat.columns = ['pwat_mean_4pm', 'pwat_min_4pm', 'pwat_max_4pm', 'pwat_sd_4pm']
    grouped_pwat = grouped_pwat.reset_index()


    agg_data = grouped_t.merge(grouped_t_2m, how = 'left', on = 'coord')
    agg_data = agg_data.merge(grouped_msl, how = 'left', on = 'coord')
    agg_data = agg_data.merge(grouped_hum_atmos, how = 'left', on = 'coord')
    agg_data = agg_data.merge(grouped_u_10m, how = 'left', on = 'coord')
    agg_data = agg_data.merge(grouped_v_10m, how = 'left', on = 'coord')
    agg_data = agg_data.merge(grouped_pwat, how = 'left', on = 'coord')
    dateval = min(dataf.date.unique())
    agg_data['date'] = dateval
    


    return agg_data

In [127]:
# download data and create a df for each day

def downloadData(file_list, dayDF2, cnt, skip):
    count1 = 0

    dayDF = None
    # dayDF2_main = None
    p = False
    for file in file_list:
        filename = dspath + file
        outfile = save_dir + os.path.basename(filename)
        print('Trying to downloading', file)
        req = requests.get(filename, cookies = ret.cookies, allow_redirects=True)
        if req.status_code != 200:
            print('File not found')
            continue
        else:
            count1+=1
            open(outfile, 'wb').write(req.content)
            file_name = save_dir + file.split('/')[-1]
            print('downloaded')
            mainDF = getData(file.split('/')[-1])
            
            if dayDF is None:
                dayDF = mainDF.copy(deep = True)
            else:
                dayDF = pd.concat([dayDF, mainDF])
            
            if count1>1:
                if cnt == 0:
                    dayDF2 = mainDF.copy(deep = True)
                else:
                    dayDF2 = pd.concat([dayDF2, mainDF])
                cnt+=1
            if count1 == 1 and cnt ==3:
                dayDF2 = pd.concat([dayDF2, mainDF])
                dayDF2_main = dayDF2.copy(deep = True)
                dayDF2 = None
                p = True
                cnt=0
                
    if dayDF is None:
        return None, None, 0, None

    aggregated = aggregateData9am(dayDF)
    if p:
        # print(dayDF2_main)
        # print(min(dayDF2_main.date.unique()))
        aggregated_4pm = aggregateData4pm(dayDF2_main)
        return aggregated, dayDF2, cnt, aggregated_4pm
        # return aggregated, dayDF2, cnt, None

    return aggregated, dayDF2, cnt, None
    # return dayDF

In [141]:
# ['grib2/2020/2020.01/fnl_20200101_12_00.grib2']
cnt_4pm = 0
df_4pm = None
mainDF_4pm = None

for filelist in mainfiles[35:]:
                
            # print(filelist)
    mainDF, df_4pm, cnt_4pm, mainDF_4pm= downloadData(filelist, df_4pm, cnt_4pm, skip = False)
    
    if mainDF is not None:
        if mainDF_4pm is None:
            mainDF_9am = mainDF.copy(deep = True)
        else:
            all_data = mainDF_9am.merge(mainDF_4pm, how = 'left', on = 'coord')
            all_data = all_data.drop(['date_x'], axis = 1)
            all_data = all_data.rename(columns = {'date_y':'date'})
            # print(all_data.columns)
            addToDb(all_data)

Trying to downloading grib1/2007/2007.02/fnl_20070228_00_00.grib1
downloaded
Trying to downloading grib1/2007/2007.02/fnl_20070228_06_00.grib1
downloaded
Trying to downloading grib1/2007/2007.02/fnl_20070228_12_00.grib1
downloaded
Trying to downloading grib1/2007/2007.02/fnl_20070228_18_00.grib1
downloaded
Trying to downloading grib1/2007/2007.03/fnl_20070301_00_00.grib1
downloaded
Trying to downloading grib1/2007/2007.04/fnl_20070430_00_00.grib1
downloaded
Trying to downloading grib1/2007/2007.04/fnl_20070430_06_00.grib1
downloaded
Trying to downloading grib1/2007/2007.04/fnl_20070430_12_00.grib1
downloaded
Trying to downloading grib1/2007/2007.04/fnl_20070430_18_00.grib1
downloaded
Trying to downloading grib1/2007/2007.05/fnl_20070501_00_00.grib1
downloaded
Trying to downloading grib1/2007/2007.06/fnl_20070630_00_00.grib1
downloaded
Trying to downloading grib1/2007/2007.06/fnl_20070630_06_00.grib1
downloaded
Trying to downloading grib1/2007/2007.06/fnl_20070630_12_00.grib1
downloaded

In [140]:
mainfiles[35]

['grib1/2007/2007.02/fnl_20070228_00_00.grib1',
 'grib1/2007/2007.02/fnl_20070228_06_00.grib1',
 'grib1/2007/2007.02/fnl_20070228_12_00.grib1',
 'grib1/2007/2007.02/fnl_20070228_18_00.grib1',
 'grib1/2007/2007.03/fnl_20070301_00_00.grib1']